# Bakery Inventory Optimization — Flour Demand Forecasting

**Goal:** Transform sales data into actionable flour demand forecasts for smarter inventory decisions.

- Data transformation: transactions → daily flour demand
- Multiple forecasting models & backtesting
- Probabilistic safety stock recommendations


## 1. Setup & Data Loading


In [33]:
# Import libraries and load data
import pandas as pd
import numpy as np
import plotly.graph_objects as go

from IPython.display import Markdown

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from prophet import Prophet
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

df = pd.read_csv('bread basket.csv')
display(df.info(), df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20507 entries, 0 to 20506
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Transaction      20507 non-null  int64 
 1   Item             20507 non-null  object
 2   date_time        20507 non-null  object
 3   period_day       20507 non-null  object
 4   weekday_weekend  20507 non-null  object
dtypes: int64(1), object(4)
memory usage: 801.2+ KB


None

,Transaction,Item,date_time,period_day,weekday_weekend
0,1,Bread,30-10-2016 09:58,morning,weekend
1,2,Scandinavian,30-10-2016 10:05,morning,weekend
2,2,Scandinavian,30-10-2016 10:05,morning,weekend
3,3,Hot chocolate,30-10-2016 10:07,morning,weekend
4,3,Jam,30-10-2016 10:07,morning,weekend


## 2. Data Preprocessing & Aggregation

- Parse timestamps, aggregate sales to daily counts per SKU
- Prepare continuous time series for modeling


In [34]:
# Parse dates, aggregate sales per item per day
df_grouped = df.copy()
df_grouped['date_time'] = pd.to_datetime(df_grouped['date_time'], dayfirst=True)
df_grouped['date_time'] = df_grouped['date_time'].dt.strftime('%d-%m-%Y')
df_grouped = df_grouped.groupby(['date_time', 'Item']).size().reset_index(name='sales')
df_grouped.head()

,date_time,Item,sales
0,01-01-2017,Bread,1
1,01-02-2017,Alfajores,3
2,01-02-2017,Baguette,3
3,01-02-2017,Bakewell,1
4,01-02-2017,Bread,33


## 3. Exploratory Data Analysis (EDA)

- Identify top-selling products and visualize demand patterns


In [35]:
# Top-selling items
sku_sales = df_grouped.groupby('Item')['sales'].sum().sort_values(ascending=False)
display(sku_sales.head())

Item
Coffee    5471
Bread     3325
Tea       1435
Cake      1025
Pastry     856
Name: sales, dtype: int64

## 4. Feature Engineering: Product Sales → Flour Usage

- Map products to estimated flour usage (kg/unit)
- Compute daily flour demand for inventory planning


In [36]:
# Map product to flour usage and compute daily demand
flour_equivalents = {
    "Bread": 0.45,   
    "Baguette": 0.35,
    "Cake": 0.30,
    "Muffin": 0.12,
    "Brownie": 0.10,
    "Pastry": 0.20,
    "Medialuna": 0.15,
    "Cookies": 0.08,
    "Scone": 0.10,
    "Empanadas": 0.18,
    "Crepes": 0.05
}

df_grouped['flour_kg'] = df_grouped['Item'].map(flour_equivalents).fillna(0)
df_grouped['flour_used'] = np.round(df_grouped['sales'] * df_grouped['flour_kg'], 2)
df_grouped = df_grouped[df_grouped['flour_used'] > 0]
df_grouped.head()

,date_time,Item,sales,flour_kg,flour_used
0,01-01-2017,Bread,1,0.45,0.45
2,01-02-2017,Baguette,3,0.35,1.05
4,01-02-2017,Bread,33,0.45,14.85
5,01-02-2017,Brownie,1,0.10,0.10
6,01-02-2017,Cake,9,0.30,2.70


## 5. Time Series Construction

- Aggregate daily flour demand, fill missing days with 0
- Prepare data for forecasting models


In [37]:
# Aggregate to daily flour demand, fill missing days

df_fc = df_grouped.groupby('date_time')['flour_used'].sum().reset_index()
df_fc['flour_used'] = np.ceil(df_fc['flour_used'])
df_fc = df_fc.rename(columns={'date_time': 'ds', 'flour_used': 'y'})
df_fc['ds'] = pd.to_datetime(df_fc['ds'], dayfirst=True)
df_fc = df_fc.sort_values('ds').set_index('ds').asfreq('D', fill_value=0).reset_index()
print(f"Total days: {(df_fc['ds'].max() - df_fc['ds'].min()).days}")
df_fc.head()

Total days: 161


,ds,y
0,2016-10-30,18.0
1,2016-10-31,18.0
2,2016-11-01,16.0
3,2016-11-02,12.0
4,2016-11-03,20.0


In [38]:
# Visualize historical flour demand
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_fc['ds'],
    y=df_fc['y'],
    mode='lines',
    name='Flour demand',
    line=dict(color="#D21E54", width=2.5),
    fill='tozeroy',
    fillcolor='rgba(210, 30, 84, 0.15)'
))

fig.update_layout(
    title=dict(text='Historical Flour Demand', font=dict(size=16, color='black')),
    xaxis_title='Date',
    yaxis_title='Flour Used (kg)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    height=450
)

fig.show()

In [39]:
# Analyze demand by day of week
df_weekly = df_fc.copy()
df_weekly['day_of_week'] = df_weekly['ds'].dt.day_name()
df_weekly['day_num'] = df_weekly['ds'].dt.dayofweek

# Define proper day order for plotting
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Calculate average demand by day of week
weekly_demand = df_weekly.groupby('day_of_week')['y'].agg(['mean', 'std']).reindex(day_order).reset_index()

# Color weekends differently
colors = ['#6A0DAD' if day not in ['Saturday', 'Sunday'] else '#D21E54' for day in weekly_demand['day_of_week']]

# Plotly bar chart
fig = go.Figure()

fig.add_trace(go.Bar(
    x=weekly_demand['day_of_week'],
    y=weekly_demand['mean'],
    name='Average Demand',
    marker_color=colors,
    marker_line_color='black',
    marker_line_width=1.2,
    error_y=dict(type='data', array=weekly_demand['std'], visible=True, color='black', thickness=1.5)
))

# Add horizontal line for average
overall_avg = weekly_demand['mean'].mean()
fig.add_hline(
    y=overall_avg,
    line_dash="dash",
    line_color="black",
    annotation_text=f"Weekly Average: {overall_avg:.1f} kg",
    annotation_position="left"
)

fig.update_layout(
    title=dict(text='Weekly Demand Pattern: Bakery Seasonality', font=dict(size=16)),
    xaxis_title='Day of Week',
    yaxis_title='Average Flour Demand (kg)',
    template='plotly_white',
    showlegend=False,
    height=450,
    hovermode='x unified'
)

fig.show()

# Print insights
weekend_avg = weekly_demand[weekly_demand['day_of_week'].isin(['Saturday', 'Sunday'])]['mean'].mean()
weekday_avg = weekly_demand[~weekly_demand['day_of_week'].isin(['Saturday', 'Sunday'])]['mean'].mean()
peak_idx = weekly_demand['mean'].idxmax()
low_idx = weekly_demand['mean'].idxmin()

print(f"  • Weekend Average: {weekend_avg:.1f} kg")
print(f"  • Weekday Average: {weekday_avg:.1f} kg")
print(f"  • Weekend vs Weekday: {((weekend_avg - weekday_avg) / weekday_avg * 100):.1f}% higher demand")
print(f"  • Peak Day: {weekly_demand.iloc[peak_idx]['day_of_week']} ({weekly_demand.iloc[peak_idx]['mean']:.1f} kg)")
print(f"  • Lowest Day: {weekly_demand.iloc[low_idx]['day_of_week']} ({weekly_demand.iloc[low_idx]['mean']:.1f} kg)")

  • Weekend Average: 19.0 kg
  • Weekday Average: 12.8 kg
  • Weekend vs Weekday: 48.7% higher demand
  • Peak Day: Saturday (23.3 kg)
  • Lowest Day: Monday (10.9 kg)


In [40]:
# --- Forecasting Model Functions ---
def fit_predict_prophet(train_df, h):
    m = Prophet(weekly_seasonality=True, yearly_seasonality=False, daily_seasonality=False, growth='linear', changepoint_prior_scale=0.05)
    m.fit(train_df)
    start_date = train_df['ds'].max() + pd.Timedelta(days=1)
    future_dates = pd.date_range(start=start_date, periods=h, freq='D')
    future = m.make_future_dataframe(periods=h, freq='D')
    fc = m.predict(future)
    fc_test = fc[['ds', 'yhat']].set_index('ds').reindex(future_dates).reset_index(drop=True)
    return fc_test['yhat'].reset_index(drop=True)

def fit_predict_sarimax(train_df, h):
    model = SARIMAX(train_df['y'], order=(1,1,1), seasonal_order=(1,1,1,7), enforce_stationarity=False, enforce_invertibility=False)
    res = model.fit(disp=False)
    preds = res.get_forecast(steps=h).predicted_mean
    preds = np.asarray(preds)
    preds = np.clip(preds, a_min=0, a_max=None)
    return pd.Series(preds).reset_index(drop=True)

def fit_predict_ets(train_df, h):
    model = ExponentialSmoothing(train_df['y'], trend=None, seasonal='add', seasonal_periods=7, initialization_method='estimated')
    res = model.fit(optimized=True, use_brute=True)
    preds = res.forecast(h)
    preds = preds.clip(lower=0)
    return pd.Series(preds).reset_index(drop=True)

def fit_predict_seasonal_naive(train_df, h, period=7):
    last_season = train_df['y'].iloc[-period:].values
    repeats = int(np.ceil(h / period))
    fc = np.tile(last_season, repeats)[:h]
    return pd.Series(fc).reset_index(drop=True)

## 6. Forecasting & Model Evaluation

- Train multiple models (ETS, SARIMAX, Prophet, Naive)
- Compare using expanding-window backtest (MAE, WAPE, sMAPE)


In [41]:
def rolling_backtest(df, model_funcs, horizon=30, n_splits=5):
    """
    Perform an expanding-window rolling backtest.

    Parameters
    - df: DataFrame with 'ds' and 'y'
    - model_funcs: dict name->callable(train_df, h)
    - horizon: forecast horizon for each fold
    - n_splits: number of folds (last n_splits horizons will be used)

    Returns
    - metrics_df: per-fold metrics
    - summary: per-model mean metrics across folds
    - last_preds: dict of predictions for the last fold
    - last_test: DataFrame of the last fold test (with 'ds' and 'y')
    """
    total_days = len(df)
    if total_days < horizon * (n_splits + 1):
        raise ValueError("Not enough data for requested n_splits and horizon")

    records = []
    last_preds = {}
    last_test = None
    EPS = 1e-8

    for i in range(n_splits):
        train_end = total_days - horizon * (n_splits - i)
        train = df.iloc[:train_end].reset_index(drop=True)
        test = df.iloc[train_end:train_end + horizon].reset_index(drop=True)
        y_true = test['y'].reset_index(drop=True)
        denom = np.sum(np.abs(y_true))

        for name, func in model_funcs.items():
            y_pred = func(train, horizon)
            y_pred = pd.Series(y_pred).reset_index(drop=True)[:len(y_true)].clip(lower=0)
            mae = mean_absolute_error(y_true, y_pred)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))
            with np.errstate(divide='ignore', invalid='ignore'):
                mape_vals = np.abs((y_true - y_pred) / y_true)
                mape = np.nanmean(np.where(np.isfinite(mape_vals), mape_vals, np.nan)) * 100

            smape = np.mean(2 * np.abs(y_pred.values - y_true.values) / (np.abs(y_true.values) + np.abs(y_pred.values) + EPS)) * 100

            wape = (np.sum(np.abs(y_true - y_pred)) / max(denom, EPS) * 100)

            records.append({
                'model': name,
                'fold': i + 1,
                'mae': mae,
                'rmse': rmse,
                'mape_%': mape,
                'wape_%': wape,
                'smape_%': smape
            })

            # capture last fold predictions
            if i == (n_splits - 1):
                last_preds[name] = y_pred
                last_test = test

    metrics_df = pd.DataFrame.from_records(records)
    summary = (
        metrics_df
        .groupby('model')[['mae', 'rmse', 'mape_%', 'wape_%', 'smape_%']]
        .mean()
        .reset_index()
    )

    return metrics_df, summary, last_preds, last_test

In [42]:
# Run backtest and show results
test_days = 14

models = {
    'Prophet': fit_predict_prophet,
    'SARIMAX': fit_predict_sarimax,
    'ETS': fit_predict_ets,
    'Seasonal Naive': fit_predict_seasonal_naive
}

metrics_df, summary_means, last_preds, last_test = rolling_backtest(df_fc, models, horizon=test_days, n_splits=5)

display(summary_means)

22:03:36 - cmdstanpy - INFO - Chain [1] start processing
22:03:36 - cmdstanpy - INFO - Chain [1] done processing
22:03:38 - cmdstanpy - INFO - Chain [1] start processing
22:03:38 - cmdstanpy - INFO - Chain [1] done processing
22:03:39 - cmdstanpy - INFO - Chain [1] start processing
22:03:39 - cmdstanpy - INFO - Chain [1] done processing
22:03:40 - cmdstanpy - INFO - Chain [1] start processing
22:03:40 - cmdstanpy - INFO - Chain [1] done processing
22:03:41 - cmdstanpy - INFO - Chain [1] start processing
22:03:41 - cmdstanpy - INFO - Chain [1] done processing


,model,mae,rmse,mape_%,wape_%,smape_%
0,ETS,2.744975,3.446308,22.892107,18.677147,19.253777
1,Prophet,3.490601,4.103505,26.512543,22.787683,25.261703
2,SARIMAX,2.866656,3.541675,23.832192,19.433562,20.092705
3,Seasonal Naive,3.257143,4.023120,25.296276,22.247200,22.497915


In [43]:
# Visualize actual vs predicted for last test period
fig = go.Figure()

# Actual demand
fig.add_trace(go.Scatter(
    x=last_test['ds'],
    y=last_test['y'],
    mode='lines+markers',
    name='Actual demand',
    line=dict(color='black', width=3),
    marker=dict(size=8, color='white', line=dict(color='black', width=2))
))

# Model predictions
palette = {
    'Prophet': '#D21E54',       
    'SARIMAX': '#6A0DAD',       
    'ETS': '#FF1493',           
    'Seasonal Naive': '#9370DB' 
}

for name, preds in last_preds.items():
    color = palette.get(name, '#888888')
    y_vals = preds.values if hasattr(preds, 'values') else preds
    fig.add_trace(go.Scatter(
        x=last_test['ds'],
        y=y_vals,
        mode='lines+markers',
        name=name,
        line=dict(color=color, width=2.5),
        marker=dict(size=6, color=color)
    ))

fig.update_layout(
    title=dict(text=f'Model Comparison: Last {len(last_test)} Days', font=dict(size=16)),
    xaxis_title='Date',
    yaxis_title='Flour Used (kg)',
    template='plotly_white',
    hovermode='x unified',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    height=450
)

fig.show()

In [44]:
# Probabilistic forecasting: Monte Carlo quantiles for inventory

def ets_bootstrap_mc(fit_model, residuals, lead_time, n_sims=10000, seed=42):
    """
    Residual-based bootstrap Monte Carlo for ETS using statsmodels.
    Returns cumulative demand over lead_time for each simulation.
    
    Parameters:
    - fit_model: fitted ETS model from statsmodels
    - residuals: extracted residuals from the model
    - lead_time: forecast horizon (days)
    - n_sims: number of simulations
    
    Returns:
    - cum_forecasts: (n_sims,) array of cumulative demands
    - day_forecasts: (n_sims, lead_time) array of daily forecasts
    """
    np.random.seed(seed)
    base_forecast = fit_model.forecast(steps=lead_time).values
    cum_forecasts = np.zeros(n_sims)
    day_forecasts = np.zeros((n_sims, lead_time))
    
    # Residual-based bootstrap: add resampled residuals to base forecast
    for sim in range(n_sims):
        # Resample residuals with replacement
        sampled_resids = np.random.choice(residuals.values, size=lead_time, replace=True)
        
        # Add resampled residuals to base forecast
        sim_forecast = base_forecast + sampled_resids
        sim_forecast = np.maximum(sim_forecast, 0)  # Non-negativity constraint
        
        day_forecasts[sim, :] = sim_forecast
        cum_forecasts[sim] = sim_forecast.sum()
        
    return cum_forecasts, day_forecasts

In [45]:
# Fit ETS model and run Monte Carlo for safety stock

model = ExponentialSmoothing(
    df_fc['y'],
    trend="add",
    seasonal="add",
    seasonal_periods=7,
    initialization_method='estimated'
    )

fit = model.fit(optimized=True, use_brute=True)

lead_time = 4

print(f'Lead time: {lead_time} days')

residuals = fit.resid.dropna()

cum_forecasts, day_forecasts = ets_bootstrap_mc(fit, residuals, lead_time=lead_time, n_sims=10000)

quantiles = [0.95, 0.98, 0.99]
quantile_values = np.quantile(cum_forecasts, quantiles)

results_df = pd.DataFrame({'Quantile': ['95%', '98%', '99%'],
                            'Cumulative Flour (kg)': quantile_values})
display(results_df)
print(f"Mean forecast: {cum_forecasts.mean():.1f} kg | Std: {cum_forecasts.std():.1f} kg")

Lead time: 4 days


,Quantile,Cumulative Flour (kg)
0,95%,50.284147
1,98%,53.375779
2,99%,55.445193


Mean forecast: 39.0 kg | Std: 7.1 kg


## 7. Inventory Recommendations: Probabilistic Safety Stock

- Use Monte Carlo quantiles to set stock for desired service level (e.g., 95%)
- Table: mean, 95%, 98%, 99% recommended stock and safety stock

In [46]:
# Compute daily quantiles
daily_quantiles = {
    'q05': np.quantile(day_forecasts, 0.05, axis=0),
    'q25': np.quantile(day_forecasts, 0.25, axis=0),
    'q50': np.quantile(day_forecasts, 0.50, axis=0),
    'q75': np.quantile(day_forecasts, 0.75, axis=0),
    'q95': np.quantile(day_forecasts, 0.95, axis=0),
}

days = list(range(1, lead_time + 1))

# ============== FIGURE 1: Fan Chart ==============
fig1 = go.Figure()

# 90% band
fig1.add_trace(go.Scatter(
    x=days + days[::-1],
    y=list(daily_quantiles['q95']) + list(daily_quantiles['q05'][::-1]),
    fill='toself',
    fillcolor='rgba(210, 30, 84, 0.2)',
    line=dict(color='rgba(255,255,255,0)'),
    name='90% of scenarios'
))

# 50% band
fig1.add_trace(go.Scatter(
    x=days + days[::-1],
    y=list(daily_quantiles['q75']) + list(daily_quantiles['q25'][::-1]),
    fill='toself',
    fillcolor='rgba(210, 30, 84, 0.35)',
    line=dict(color='rgba(255,255,255,0)'),
    name='50% of scenarios'
))

# Median line
fig1.add_trace(go.Scatter(
    x=days,
    y=daily_quantiles['q50'],
    mode='lines+markers',
    name='Median',
    line=dict(color='#D21E54', width=2.5),
    marker=dict(size=8, color='#D21E54')
))

fig1.update_layout(
    template='plotly_white',
    title=dict(text='Daily Demand: Probabilistic Forecast', font=dict(size=14, weight='bold')),
    height=450,
    xaxis_title='Day',
    yaxis_title='Daily Flour Demand (kg)',
    legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.01),
    hovermode='x unified'
)

# ============== FIGURE 2: Cumulative Distribution ==============
fig2 = go.Figure()

# Histogram
fig2.add_trace(go.Histogram(
    x=cum_forecasts,
    nbinsx=60,
    name='Simulations',
    marker_color='#D21E54',
    marker_line_color='white',
    marker_line_width=0.5,
    opacity=0.7
))

# Service level lines
q95 = np.quantile(cum_forecasts, 0.95)
q98 = np.quantile(cum_forecasts, 0.98)
q99 = np.quantile(cum_forecasts, 0.99)
mean_val = cum_forecasts.mean()

# Define service levels with positions for annotations
service_levels = [
    {'val': mean_val, 'color': 'black', 'dash': 'solid', 'label': 'Mean', 'y_pos': 0.95},
    {'val': q95, 'color': '#1f77b4', 'dash': 'dash', 'label': '95%', 'y_pos': 0.85},
    {'val': q98, 'color': '#ff7f0e', 'dash': 'dash', 'label': '98%', 'y_pos': 0.75},
    {'val': q99, 'color': '#2ca02c', 'dash': 'dash', 'label': '99%', 'y_pos': 0.65},
]

for sl in service_levels:
    fig2.add_vline(
        x=sl['val'], 
        line=dict(color=sl['color'], width=2.5, dash=sl['dash'])
    )
    
    fig2.add_annotation(
        x=sl['val'],
        y=sl['y_pos'],
        yref='paper',
        text=f"<b>{sl['label']}</b>: {sl['val']:.0f} kg",
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowcolor=sl['color'],
        ax=30,
        ay=0,
        font=dict(size=11, color=sl['color']),
        bgcolor='rgba(255,255,255,0.8)',
        bordercolor=sl['color'],
        borderwidth=1,
        borderpad=3
    )

fig2.update_layout(
    template='plotly_white',
    title=dict(text=f'{lead_time}-Day Cumulative Demand Distribution', font=dict(size=14, weight='bold')),
    height=450,
    xaxis_title='Cumulative Flour Demand (kg)',
    yaxis_title='Frequency (# of simulations)',
    showlegend=False,
    hovermode='x unified'
)

fig1.show()
fig2.show()

display(Markdown(f"### Recommended Reorder Point (ROP) Table — {lead_time}-Day Lead Time"))

summary_table = pd.DataFrame({
    'Confidence Level': ['Mean (50%)', '95%', '98%', '99%'],
    'Recommended Stock (kg)': [f"{mean_val:.1f}", f"{q95:.1f}", f"{q98:.1f}", f"{q99:.1f}"],
    'Safety Stock (vs Mean)': [f"{0:.1f}", f"{q95 - mean_val:.1f}", f"{q98 - mean_val:.1f}", f"{q99 - mean_val:.1f}"]
})
display(summary_table)

### Recommended Reorder Point (ROP) Table — 4-Day Lead Time

,Confidence Level,Recommended Stock (kg),Safety Stock (vs Mean)
0,Mean (50%),39.0,0.0
1,95%,50.3,11.3
2,98%,53.4,14.4
3,99%,55.4,16.5


## 8. Conclusion

- This project transformed raw transaction data into daily flour demand, enabling data-driven inventory management for bakeries.
- Multiple time-series models were evaluated, and a probabilistic approach was used to recommend inventory levels for a chosen lead time.
- For a lead time of 4 days, the final summary table shows the mean forecast and the stock required to cover 95%, 98%, and 99% of simulated demand scenarios.
- Higher service levels require more safety stock, helping you avoid stockouts at the cost of holding more inventory. Choose the level that best fits your business risk tolerance.
- This approach helps reduce waste, avoid lost sales, and optimize inventory for real business impact.
